In [17]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Importing API key
from config_google import gkey

In [18]:
# Storing csv created in Weather API assignment into a DataFrame
cities_data_df = pd.read_csv("weathercsv.csv")
cities_data_df.head()

,Cities,Country,Date,Latitude,Longitude,Temp,Humidity,Windspeed,Cloudiness
0,bethel,US,1612982149,41.3712,-73.4140,30.99,50,3.44,1
1,malanje,AO,1612981894,-9.5402,16.3410,66.87,93,5.46,88
2,tadine,NC,1612982149,-21.5500,167.8833,76.48,84,6.13,11
3,cidreira,BR,1612982149,-30.1811,-50.2056,77.52,74,13.29,100
4,qaanaaq,GL,1612982149,77.4840,-69.3632,3.07,71,11.79,99


### Heatmap of Humidity


In [19]:
# Configuring gmaps
gmaps.configure(api_key=gkey)

In [27]:
# Heatmap of humidity
locations = cities_data_df[["Latitude", "Longitude"]]
humidity = cities_data_df["Humidity"]
fig = gmaps.figure()
heat_Map = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, \
                                 max_intensity=300, point_radius=5)

fig.add_layer(heat_Map)

fig

Figure(layout=FigureLayout(height='420px'))

### Creating cities df that is narrowed down to weather criteria

In [21]:
narrowed_city_df = cities_data_df.loc[(cities_data_df["Temp"] < 80) & (cities_data_df["Temp"] > 70) \
                                    & (cities_data_df["Windspeed"] < 10) \
                                    & (cities_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,Cities,Country,Date,Latitude,Longitude,Temp,Humidity,Windspeed,Cloudiness
8,hermanus,ZA,1612982150,-34.4187,19.2345,71.01,76,3.00,0
61,bilma,NE,1612982159,18.6853,12.9164,73.62,17,7.74,0
152,tessalit,ML,1612982178,20.1986,1.0114,76.15,16,9.42,0
156,biltine,TD,1612982178,14.5333,20.9167,73.09,20,8.97,0
191,norsup,VU,1612982185,-16.0667,167.3833,79.54,76,6.46,0
258,port augusta,AU,1612982196,-32.5000,137.7667,78.01,46,5.75,0
287,port blair,IN,1612982201,11.6667,92.7500,76.59,69,9.53,0
361,thamaga,BW,1612982213,-24.6701,25.5397,77.00,69,3.44,0
368,kidal,ML,1612982214,18.4411,1.4078,75.78,16,5.23,0
372,magaria,NE,1612982215,12.9983,8.9099,75.79,15,8.68,0


### Heat Map


In [22]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["Cities", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,Cities,Country,Latitude,Longitude,Hotel Name
8,hermanus,ZA,-34.4187,19.2345,
61,bilma,NE,18.6853,12.9164,
152,tessalit,ML,20.1986,1.0114,
156,biltine,TD,14.5333,20.9167,
191,norsup,VU,-16.0667,167.3833,
258,port augusta,AU,-32.5000,137.7667,
287,port blair,IN,11.6667,92.7500,
361,thamaga,BW,-24.6701,25.5397,
368,kidal,ML,18.4411,1.4078,
372,magaria,NE,12.9983,8.9099,


In [23]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": gkey
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,Cities,Country,Latitude,Longitude,Hotel Name
8,hermanus,ZA,-34.4187,19.2345,
61,bilma,NE,18.6853,12.9164,
152,tessalit,ML,20.1986,1.0114,
156,biltine,TD,14.5333,20.9167,
191,norsup,VU,-16.0667,167.3833,
258,port augusta,AU,-32.5000,137.7667,
287,port blair,IN,11.6667,92.7500,
361,thamaga,BW,-24.6701,25.5397,
368,kidal,ML,18.4411,1.4078,
372,magaria,NE,12.9983,8.9099,


In [24]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [25]:
# Add marker layer on top of the heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))